# Getting Soil Grid Data

In [ ]:
import pandas as pd
import requests
import json
import time
import random
import os
import concurrent.futures
from tqdm import tqdm
import threading

# Add a lock to prevent race conditions when saving data
save_lock = threading.Lock()

def get_soilgrids_point(lon, lat, point_idx, properties=None, max_retries=3):
    """Get SoilGrids data for a single point with correct field mapping"""
    if properties is None:
        properties = ['soc', 'clay', 'sand', 'silt', 'bdod', 'phh2o']
        
    url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    params = {
        'lon': lon,
        'lat': lat,
        'property': properties,
        'depth': ['0-5cm', '5-15cm', '15-30cm'],
        'value': ['mean']
    }
    
    for retry in range(max_retries):
        try:
            response = requests.get(url, params=params)
            
            # Handle rate limiting
            if response.status_code == 429:
                wait_time = 15 + random.random() * 15
                #print(f"Rate limited for point {point_idx}, waiting {wait_time:.1f} seconds")
                time.sleep(wait_time)
                continue
                
            if response.status_code == 200:
                data = response.json()
                
                # Start with basic info
                result = {'point_index': point_idx, 'lon': lon, 'lat': lat}
                
                # Extract data using the correct field structure
                if 'properties' in data and 'layers' in data['properties']:
                    for layer in data['properties']['layers']:
                        # Get property name
                        prop_name = layer.get('name', 'unknown')
                        
                        for depth in layer.get('depths', []):
                            # Get depth label (which is the string format we need)
                            depth_label = depth.get('label', 'unknown')
                            
                            # Clean the depth label for column naming
                            clean_depth = depth_label.replace('-', '_to_')
                            
                            # Extract values
                            for value_type, value in depth.get('values', {}).items():
                                column_name = f"{prop_name}_{clean_depth}_{value_type}"
                                result[column_name] = value
                
                # Debug print to verify data is being captured correctly
                #print(f"Retrieved data for point {point_idx}: {lon}, {lat}")
                return result
            else:
                #print(f"Error for point {point_idx}: Status code {response.status_code}")
                if retry < max_retries - 1:
                    wait_time = 10 * (retry + 1)
                    #print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    return {'point_index': point_idx, 'lon': lon, 'lat': lat, 
                            'error': f"Status {response.status_code}"}
        
        except Exception as e:
            #print(f"Exception for point {point_idx}: {str(e)}")
            if retry < max_retries - 1:
                wait_time = 10 * (retry + 1)
                #print(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                return {'point_index': point_idx, 'lon': lon, 'lat': lat, 
                        'error': f"Exception: {str(e)}"}
    
    return {'point_index': point_idx, 'lon': lon, 'lat': lat, 
            'error': "Max retries reached"}

def process_point(args):
    """Wrapper function for concurrent processing"""
    lon, lat, idx, properties = args
    # Add jitter to avoid all workers hitting the API simultaneously
    time.sleep(random.random() * 2)
    return get_soilgrids_point(lon, lat, idx, properties)

def save_checkpoint(results, filename, verbose=True):
    """Save results to a checkpoint file using a lock to prevent race conditions"""
    with save_lock:
        try:
            df_results = pd.DataFrame(results)
            # First write to a temporary file, then rename to avoid partial writes
            temp_file = f"{filename}.temp"
            df_results.to_csv(temp_file, index=False)
            os.replace(temp_file, filename)
            if verbose:
                print(f"Saved checkpoint with {len(results)} points to {filename}")
        except Exception as e:
            print(f"Error saving checkpoint: {str(e)}")

def get_soilgrids_parallel(coordinates_df, num_workers=4, lon_col='GPS_LONG', lat_col='GPS_LAT', 
                           properties=None, cache_file='soilgrids_parallel.csv',
                           checkpoint_interval=10, debug=True):
    """
    Retrieve soil data for multiple points in parallel using multiple workers
    
    Args:
        coordinates_df: DataFrame with coordinates
        num_workers: Number of parallel workers (default: 4)
        lon_col: Column name for longitude
        lat_col: Column name for latitude
        properties: List of SoilGrids properties to retrieve
        cache_file: Output file name
        checkpoint_interval: Save intermediate results every N points
        debug: Enable additional debug output
    """
    if properties is None:
        properties = ['soc', 'clay', 'sand', 'silt', 'bdod', 'phh2o']
    
    # Print the input data to verify it's correct
    if debug:
        print("Input coordinate data sample:")
        print(coordinates_df.head())
        print(f"Longitude column: {lon_col}, Latitude column: {lat_col}")
    
    # Check for existing cache to resume from
    results = []
    
    if os.path.exists(cache_file):
        try:
            existing_df = pd.read_csv(cache_file)
            if len(existing_df) > 0:
                results = existing_df.to_dict('records')
                processed_indices = set(existing_df['point_index'].unique())
                print(f"Found {len(processed_indices)} already processed points in {cache_file}")
                coordinates_df = coordinates_df[~coordinates_df.index.isin(processed_indices)]
                print(f"Remaining points to process: {len(coordinates_df)}")
        except Exception as e:
            print(f"Error reading existing cache: {str(e)}. Starting from scratch.")
    
    if len(coordinates_df) == 0:
        print("All points already processed!")
        return pd.DataFrame(results)
    
    # Prepare arguments for parallel processing
    args_list = []
    for idx, row in coordinates_df.iterrows():
        # Verify and clean coordinate values
        try:
            lon = float(row[lon_col])
            lat = float(row[lat_col])
            args_list.append((lon, lat, idx, properties))
            if debug and len(args_list) <= 5:
                print(f"Prepared point {idx}: lon={lon}, lat={lat}")
        except (ValueError, TypeError) as e:
            print(f"Error with coordinates at index {idx}: {e}")
            print(f"Row data: {row}")
    
    print(f"Processing {len(args_list)} points with {num_workers} workers")
    
    completed_count = 0
    
    # Use ThreadPoolExecutor for parallel HTTP requests
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        # Submit all tasks
        future_to_args = {executor.submit(process_point, args): args for args in args_list}
        
        # Use tqdm for a progress bar
        for future in tqdm(concurrent.futures.as_completed(future_to_args), total=len(args_list)):
            args = future_to_args[future]
            point_idx = args[2]
            
            try:
                result = future.result()
                if result:
                    results.append(result)
                    completed_count += 1
                    
                    # Save intermediate results periodically
                    if completed_count % checkpoint_interval == 0:
                        save_checkpoint(results, cache_file)
                
            except Exception as e:
                print(f"\nError processing point {point_idx}: {str(e)}")
    
    # Save final results
    save_checkpoint(results, cache_file, verbose=False)
    
    # Verify the final output
    try:
        final_df = pd.read_csv(cache_file)
        print(f"Final output has {len(final_df)} rows and {len(final_df.columns)} columns")
        print("Column names:", final_df.columns.tolist())
        print("First few rows:")
        print(final_df.head())
    except Exception as e:
        print(f"Error verifying final output: {str(e)}")
    
    return pd.DataFrame(results)

# Example usage:
# df = pd.read_csv('coordinates.csv', index_col=0)  # Set the first column as index if that's your point_index
# results = get_soilgrids_parallel(df, num_workers=4)

In [ ]:
# Load your data
target_raw = pd.read_csv('data/France_lab.csv')
long_lat = target_raw[['GPS_LONG', 'GPS_LAT']]
get_soilgrids_parallel(long_lat, num_workers=4, properties=['soc', 'clay', 'sand', 'silt', 'bdod', 'phh2o'], debug = False)

Found 30 already processed points in soilgrids_parallel.csv
Remaining points to process: 2777
Processing 2777 points with 4 workers


  0%|          | 10/2777 [00:50<5:09:59,  6.72s/it]

Saved checkpoint with 40 points to soilgrids_parallel.csv


  1%|          | 20/2777 [01:04<56:41,  1.23s/it]  

Saved checkpoint with 50 points to soilgrids_parallel.csv


...

 22%|██▏       | 609/2777 [29:26<1:15:41,  2.09s/it]

Saved checkpoint with 640 points to soilgrids_parallel.csv


 22%|██▏       | 620/2777 [29:33<27:17,  1.32it/s]  

Saved checkpoint with 650 points to soilgrids_parallel.csv


 23%|██▎       | 627/2777 [30:28<1:44:29,  2.92s/it]
